运行前先去右边的 `Settings` 里面将 `Accelerator` 选择为 `GPU`！

否则会运行得比较慢并且报错

不运行的时候可以关了GPU节省免费时间（

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 数据清洗与修改

详情见GitHub上ipynbs/visualization.ipynb

其中有详细的解释。

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
# 用均值填充NaN
def mean_replace(df, columnName):
    df_mean = df.copy()
    impute_mean = SimpleImputer()
    raw_data = df_mean[columnName].values.reshape(-1, 1)
    new_data = impute_mean.fit_transform(raw_data)
    df_mean[columnName] = new_data
    return df_mean

# 用0填充NaN
def zero_replace(df, columnName):
    # 用0填充年龄的缺失值
    df0 = df.copy() # 复制原数据，避免原数据被覆盖

    # 实例化
    impute_0 = SimpleImputer(strategy='constant', fill_value=0)

    # 去除Age属性的原始数据，并通过values转化为一维数组，在通过reshape变为二维数组
    # 因为sklearn中传到数据必须是二维的
    raw_data = df0[columnName].values.reshape(-1, 1)

    # fit_transform()一步到位，返回填充后的数据
    new_data = impute_0.fit_transform(raw_data)

    # 用新数据替换原数据
    df0[columnName] = new_data
    
    return df0

# 用0填充val == target
def zero_replace_sp(df, columnName, target):
    # 用0填充年龄的缺失值
    df0 = df.copy() # 复制原数据，避免原数据被覆盖
    df0.loc[df0[columnName] == target, columnName] = 0
    return df0

# 用中位数处理NaN
def median_replace(df, columnName):
    df_median = df.copy()
    impute_median = SimpleImputer(strategy='median')
    raw_data = df_median[columnName].values.reshape(-1, 1)
    new_data = impute_median.fit_transform(raw_data)
    df_median[columnName] = new_data
    return df_median

def standardlize(df, columnName):
    data = df[columnName]
    return (data-data.mean())/(data.std()) 

def manip_data(df):
    df = df[~df["age"].isin([0])]

    # 用中位数填充月收入
    df = median_replace(df, "MonthlyIncome")
    # 用0填充家人
    df = zero_replace(df, "NumberOfDependents")
    
    # 用0填充欠款逾期次数
    #  df = df[~df["NumberOfTimes90DaysLate"].isin([96, 98])]
    for col in ("NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime60-89DaysPastDueNotWorse", "NumberOfTimes90DaysLate"):
        df = zero_replace_sp(df, col, 96)
        df = zero_replace_sp(df, col, 98)
    
    #     df.drop(df[np.isnan(df['MonthlyIncome'])].index, inplace=True)
    df["MonthlyIncome"] = df["MonthlyIncome"].apply(np.log1p)
    
    # log of RevolvingUtilizationOfUnsecuredLines
    df["RevolvingUtilizationOfUnsecuredLines"] = df["RevolvingUtilizationOfUnsecuredLines"].apply(np.log1p)
    
    # 全部归一化
    for columnName in df.columns.tolist():
        if columnName in ("id", "SeriousDlqin2yrs"):
            continue
        df[columnName] = standardlize(df, columnName)
        
    # 增加YoungAge与OldAge两个特征
    df['YoungAge'] = [1 if x < 21 else 0 for x in df['age']]
    df['OldAge'] = [1 if x > 65 else 0 for x in df['age']]
        
    
    return df

# 数据读入

我们在这里还需要作数据清洗与数据修改

In [ ]:
train_data = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-training.csv")
train_data.columns = ["id"] + train_data.columns.tolist()[1:]
train_data = manip_data(train_data)
# train_data.fillna(0, inplace=True) # 把部分列，比如MonthlyIncome列的NaN换为0
train_data.head(10)

In [ ]:
selectedColumns = ["RevolvingUtilizationOfUnsecuredLines", "age", "NumberOfTime30-59DaysPastDueNotWorse", "NumberOfTime60-89DaysPastDueNotWorse", "NumberOfTimes90DaysLate", "NumberOfOpenCreditLinesAndLoans", "MonthlyIncome", "NumberRealEstateLoansOrLines"]

In [ ]:
train_x = np.array(train_data[selectedColumns])
train_y = np.array(train_data.SeriousDlqin2yrs)
train_y=train_y.ravel()
train_x

In [ ]:
test_data = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-test.csv")
test_data.columns = ["id"] + test_data.columns.tolist()[1:]
test_data = manip_data(test_data)
test_data.fillna(0, inplace=True) # 把SeriousDlqin2yrs列的NaN替换为0
test_data.head(10)

In [ ]:
test_x = np.array(test_data[selectedColumns])
test_ans=pd.read_csv("/kaggle/input/GiveMeSomeCredit/sampleEntry.csv")
test_ans.head()

In [ ]:
test_y=np.array(test_ans.Probability)
test_y=test_y.ravel()

# Training with PyTorch

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable
import torch.nn.functional as f
import matplotlib.pylab as plt
import torchvision.transforms as transforms
use_gpu = torch.cuda.is_available()
device = torch.device("cuda" if use_gpu else "cpu")

In [ ]:
train_x=torch.from_numpy(train_x)
train_y=torch.from_numpy(train_y)
test_x=torch.from_numpy(test_x)
test_y=torch.from_numpy(test_y)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
if use_gpu:
    train_x,train_y = Variable(train_x.cuda()).double(), Variable(train_y.cuda()).double()
    test_x,teat_y = Variable(test_x.cuda()).double(),Variable(test_y.cuda()).double()
else:
    train_x,train_y = Variable(train_x.cuda()).double(), Variable(train_y.cuda()).double()
    test_x = Variable(test_x.cuda()).double()

ANN网络

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, n_in, n_h1, n_h2, n_out):
        super(Net, self).__init__()
        self.fc = torch.nn.Sequential(
            nn.Linear(n_in, n_h1),
            nn.Tanh(),
            nn.Linear(n_h1, n_h2),
            nn.ReLU(),
            nn.Linear(n_h2, n_out),
        )
    def forward(self, x):
        return self.fc(x)
    
# net = Net(len(selectedColumns), 800, 400, 1)
net = Net(len(selectedColumns), 800, 50, 1)
if use_gpu: 
    net = net.cuda()
net = net.double()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
loss_func = nn.MSELoss()

net.to(device)
train_x,train_y = train_x.to(device),train_y.to(device)
test_x,test_y= test_x.to(device),test_y.to(device)

In [ ]:
for epoch in range(10000):
    train_prediction = net(train_x)
    train_prediction = train_prediction.squeeze()
    train_loss = loss_func(train_prediction, train_y)
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()
    
    if epoch%10==0:
        test_prediction=net(test_x)
        test_prediction = test_prediction.squeeze()
        test_loss=loss_func(test_prediction,test_y)
        print('[INFO] train_loss is %.4f'%train_loss.cpu().data.numpy())
        print('[INFO] test_loss is %.4f'%test_loss.cpu().data.numpy())

In [ ]:
# 存储结果
test_prediction = net(test_x)
test_prediction = torch.clamp(test_prediction, min=0.0, max=1.0)

In [ ]:
if use_gpu:
    output = pd.DataFrame({'Id': [int(x) for x in test_data.values[:,0].tolist()], 'Probability': test_prediction.cuda().data.cpu().numpy().ravel()})
else:
    output = pd.DataFrame({'Id': [int(x) for x in test_data.values[:,0].tolist()], 'Probability': test_prediction})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

之后就是提交了，不知道怎么提交的话还是问一下吧

不太好描述